# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [54]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [57]:
import warnings

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

In [58]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [59]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

google_search_data_df = load_dataset(context, 'raw/google_search_data')
product_manufacturer_list_df = load_dataset(context, 'raw/product_manufacturer_list')
sales_data_df = load_dataset(context, 'raw/sales_data')
theme_list_df = load_dataset(context, 'raw/theme_list')
theme_product_list_df = load_dataset(context, 'raw/theme_product_list')
social_media_data_df = load_dataset(context, 'raw/social_media_data')

# orders_df = load_dataset(context,'raw/orders')

['/raw/google_search_data',
 '/raw/product_manufacturer_list',
 '/raw/sales_data',
 '/raw/theme_list',
 '/raw/theme_product_list',
 '/raw/social_media_data',
 '/raw/orders',
 '/cleaned/prod_df_clean',
 '/cleaned/google_search_data_df_clean',
 '/cleaned/sales_data_df_clean',
 '/cleaned/theme_list_df_clean',
 '/cleaned/theme_product_list_df_clean',
 '/cleaned/social_media_data_df_clean',
 '/cleaned/orders',
 '/processed/social_google_sales',
 '/train/features',
 '/train/target',
 '/test/features',
 '/test/target']


In [60]:
# to standardize the date format in every dataset
from datetime import datetime
def standardize_date(date_str):
    # List of possible date formats
    date_formats = ['%m/%d/%Y', '%m-%d-%Y', '%d/%m/%Y', '%Y-%m-%d']

    # Iterate through the formats and try to parse the date
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt).strftime('%d-%m-%Y')
        except ValueError:
            pass

    # If no valid format is found, return None or the original string, depending on your preference
    return None

In [61]:
google_search_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181565 entries, 0 to 181564
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   date          181565 non-null  object
 1   platform      181565 non-null  object
 2   searchVolume  181565 non-null  int64 
 3   Claim_ID      181565 non-null  int64 
 4   week_number   181565 non-null  int64 
 5   year_new      181565 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 8.3+ MB


In [62]:
# removed some columns which have all null values
# product_manufacturer_list_df=product_manufacturer_list_df[['PRODUCT_ID','Vendor']]
product_manufacturer_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67175 entries, 0 to 67174
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PRODUCT_ID  67175 non-null  int64  
 1   Vendor      67175 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   Unnamed: 5  0 non-null      float64
 6   Unnamed: 6  0 non-null      float64
dtypes: float64(5), int64(1), object(1)
memory usage: 3.6+ MB


In [63]:
sales_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4526182 entries, 0 to 4526181
Data columns (total 5 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   system_calendar_key_N  int64  
 1   product_id             int64  
 2   sales_dollars_value    float64
 3   sales_units_value      int64  
 4   sales_lbs_value        int64  
dtypes: float64(1), int64(4)
memory usage: 172.7 MB


In [64]:
theme_list_df.head()

,CLAIM_ID,Claim Name
0,0,No Claim
1,8,low carb
2,15,beans
3,16,cocoa
4,26,vanilla


In [65]:
theme_product_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91485 entries, 0 to 91484
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   PRODUCT_ID  91485 non-null  int64
 1   CLAIM_ID    91485 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB


In [66]:
social_media_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533390 entries, 0 to 533389
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Theme Id        314879 non-null  float64
 1   published_date  533390 non-null  object 
 2   total_post      533390 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 12.2+ MB


# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


## 2.1 Clean individual tables 

### product manufacturer list 


In [67]:
# product_manufacturer_list_df=product_manufacturer_list_df[['PRODUCT_ID','Vendor']]
prod_df_clean = (

    product_manufacturer_list_df
    # while iterating on testing, it's good to copy the dataset(or a subset)
    # as the following steps will mutate the input dataframe. The copy should be
    # removed in the production code to avoid introducing perf. bottlenecks.
    .copy()

    # set dtypes : nothing to do here
    .passthrough()

    .transform_columns(['Vendor'], string_cleaning, elementwise=False)
    
    .replace({'': np.NaN})
    
    # ensure that the key column does not have duplicate records
    .remove_duplicate_rows(col_names=['PRODUCT_ID'], keep_first=True)
    
    # clean column names (comment out this line while cleaning data above)
    .clean_names(case_type='snake')
)
prod_df_clean.head()

,product_id,vendor,unnamed_2,unnamed_3,unnamed_4,unnamed_5,unnamed_6
0,1,Others,NaN,NaN,NaN,NaN,NaN
1,2,Others,NaN,NaN,NaN,NaN,NaN
2,3,Others,NaN,NaN,NaN,NaN,NaN
3,4,Others,NaN,NaN,NaN,NaN,NaN
4,5,Others,NaN,NaN,NaN,NaN,NaN


In [68]:
prod_df_clean['vendor'].value_counts()

Others           28335
Private Label    21539
A                 5584
B                 5371
D                 2780
H                 1733
F                 1379
G                  230
E                  224
Name: vendor, dtype: int64

### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [69]:
save_dataset(context, prod_df_clean, 'cleaned/prod_df_clean')

# Sales data


In [70]:
sales_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4526182 entries, 0 to 4526181
Data columns (total 5 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   system_calendar_key_N  int64  
 1   product_id             int64  
 2   sales_dollars_value    float64
 3   sales_units_value      int64  
 4   sales_lbs_value        int64  
dtypes: float64(1), int64(4)
memory usage: 172.7 MB


In [71]:
# product_manufacturer_list_df=product_manufacturer_list_df[['PRODUCT_ID','Vendor']]
sales_data_df_clean = (

    sales_data_df
    # while iterating on testing, it's good to copy the dataset(or a subset)
    # as the following steps will mutate the input dataframe. The copy should be
    # removed in the production code to avoid introducing perf. bottlenecks.
    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    .drop_duplicates()
    .change_type(['sales_dollars_value'], np.int64)
    .clean_names(case_type='snake')
)
sales_data_df_clean['system_calendar_key_n'] = pd.to_datetime(sales_data_df_clean['system_calendar_key_n'], format='%Y%m%d')

sales_data_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4526182 entries, 0 to 4526181
Data columns (total 5 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   system_calendar_key_n  datetime64[ns]
 1   product_id             int64         
 2   sales_dollars_value    int64         
 3   sales_units_value      int64         
 4   sales_lbs_value        int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 207.2 MB


In [72]:
sales_data_df_clean['system_calendar_key_n'].min()

Timestamp('2016-01-09 00:00:00')

In [73]:
save_dataset(context, sales_data_df_clean, 'cleaned/sales_data_df_clean')

# Theme list


In [74]:
theme_list_df_clean = (

    theme_list_df

    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    .drop_duplicates()
    .clean_names(case_type='snake')
)

theme_list_df_clean.head()

,claim_id,claim_name
0,0,No Claim
1,8,low carb
2,15,beans
3,16,cocoa
4,26,vanilla


In [75]:
save_dataset(context, theme_list_df_clean, 'cleaned/theme_list_df_clean')

In [76]:
theme_list_df_clean[theme_list_df_clean['claim_name'] == 'convenience - easy-to-prepare']

,claim_id,claim_name
23,142,convenience - easy-to-prepare


# Theme product list

In [77]:
theme_product_list_df_clean = (

    theme_product_list_df

    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    .drop_duplicates()
    .clean_names(case_type='snake')
)

theme_product_list_df_clean.head()

,product_id,claim_id
0,26,8
1,29,8
2,48,81
3,50,81
4,74,227


In [78]:
theme_product_list_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91485 entries, 0 to 91484
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  91485 non-null  int64
 1   claim_id    91485 non-null  int64
dtypes: int64(2)
memory usage: 2.1 MB


In [79]:
save_dataset(context, theme_product_list_df_clean, 'cleaned/theme_product_list_df_clean')

# Google search

In [80]:
google_search_data_df_clean = (

    google_search_data_df

    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    .drop_duplicates()
    .clean_names(case_type='snake')
)
google_search_data_df_clean = google_search_data_df_clean.groupby(['date','claim_id','platform','year_new','week_number'], as_index = False)['search_volume'].sum()
# year_new to year in data comparison
google_search_data_df_clean['date'] = pd.to_datetime(google_search_data_df_clean['date'])
google_search_data_df_clean['year'] = google_search_data_df_clean['date'].dt.year
google_search_data_df_clean.head()

Parsing '13-01-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2016' in DD/MM/YYYY forma

,date,claim_id,platform,year_new,week_number,search_volume,year
0,2014-01-01,8,google,2014,1,6613,2014
1,2014-01-01,39,google,2014,1,181,2014
2,2014-01-01,75,google,2014,1,135,2014
3,2014-01-01,81,google,2014,1,1257,2014
4,2014-01-01,100,google,2014,1,2636,2014


In [81]:
# for year 2016 and 2017
google_search_data_df_clean_1617 = google_search_data_df_clean[google_search_data_df_clean['year'].isin([2016,2017])]

In [82]:
# year_new to year in data comparison
google_search_data_df_clean['year'].max()


2019

In [83]:
save_dataset(context, google_search_data_df_clean, 'cleaned/google_search_data_df_clean')

# Social media

In [84]:
social_media_data_df_clean = (

    social_media_data_df

    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    .drop_duplicates()

    .clean_names(case_type='snake')
    .dropna(subset=['theme_id'], how='any')        # there are some nan values , those are removed
    .change_type(['theme_id'], np.int64)
    
)
social_media_data_df_clean.dropna(subset=['theme_id'], inplace=True)
social_media_data_df_clean['published_date'] = social_media_data_df_clean['published_date'].apply(standardize_date)
# for a theme_id and a published id there are 2 rows in many cases 
social_media_data_df_clean = social_media_data_df_clean.groupby(['theme_id','published_date'], as_index = False)['total_post'].sum()
social_media_data_df_clean['published_date'] = pd.to_datetime(social_media_data_df_clean['published_date'])
social_media_data_df_clean['year'] = social_media_data_df_clean['published_date'].dt.year
social_media_data_df_clean.head()

Parsing '13-01-2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-02-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
Parsing '13-03-2016' in DD/MM/YYYY forma

,theme_id,published_date,total_post,year
0,8,2016-01-01,115,2016
1,8,2017-01-01,278,2017
2,8,2018-01-01,506,2018
3,8,2019-01-01,617,2019
4,8,2016-01-02,243,2016


In [85]:
social_media_data_df_clean['published_date'].max()
social_media_data_df_clean_16 = social_media_data_df_clean[social_media_data_df_clean['year'].isin([2016])]
social_media_data_df_clean_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70638 entries, 0 to 311680
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   theme_id        70638 non-null  int64         
 1   published_date  70638 non-null  datetime64[ns]
 2   total_post      70638 non-null  int64         
 3   year            70638 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.7 MB


In [86]:
save_dataset(context, social_media_data_df_clean, 'cleaned/social_media_data_df_clean')

In [87]:
social_set = set(social_media_data_df_clean['theme_id'].unique().tolist())

In [88]:
theme_set = set(theme_list_df_clean['claim_id'].unique().tolist())

In [89]:
social_set - theme_set

set()

In [90]:
# sales_data_df_clean['product_id'].nunique()
# sales_data_df_clean.tail(100)

# ['system_calendar_key_N','product_id']    are PK for sales data
sales_data_df_clean[['system_calendar_key_n','product_id']].value_counts()
sales_data_df_clean[['system_calendar_key_n','product_id']].nunique()

system_calendar_key_n      196
product_id               42616
dtype: int64

In [91]:
# CLAIM_ID   is PK for theme list 
theme_list_df_clean['claim_id'].value_counts()
theme_list_df_clean['claim_id'].info()

<class 'pandas.core.series.Series'>
Int64Index: 208 entries, 0 to 207
Series name: claim_id
Non-Null Count  Dtype
--------------  -----
208 non-null    int64
dtypes: int64(1)
memory usage: 3.2 KB


In [92]:
theme_product_list_df_clean['product_id'].value_counts()
# theme_product_list_df_clean['PRODUCT_ID'].nunique()

28136    14
18693    14
27053    14
27051    14
27076    14
         ..
44591     1
2476      1
9221      1
50985     1
34907     1
Name: product_id, Length: 57317, dtype: int64

In [93]:
prod_df_clean['vendor'].nunique()
prod_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67175 entries, 0 to 67174
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  67175 non-null  int64  
 1   vendor      67175 non-null  object 
 2   unnamed_2   0 non-null      float64
 3   unnamed_3   0 non-null      float64
 4   unnamed_4   0 non-null      float64
 5   unnamed_5   0 non-null      float64
 6   unnamed_6   0 non-null      float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.1+ MB


In [94]:
col = sales_data_df_clean['sales_dollars_value']/sales_data_df_clean['sales_units_value']
col.head(30)

0     14.911135
1      6.462940
2     16.227273
3     11.402565
4      7.173321
5     12.194430
6     15.533411
7     11.497024
8     17.078524
9     18.019097
10    18.079526
11    16.823923
12    27.923107
13    16.285304
14    16.535211
15    18.000000
16    17.902439
17    15.392344
18     6.956897
19     6.810345
20     6.736842
21     6.609195
22     6.924528
23     0.625000
24     6.241379
25     8.362069
26     1.000000
27     9.199377
28    33.488636
29     3.454545
dtype: float64

## 2.2 Create consolidated features table

Here we take the cleaned dataframes and merge them to form the consolidated table.



In [95]:
prod_sales = pd.merge(prod_df_clean, sales_data_df_clean, how='inner', on='product_id', validate='1:m')
merge_info(prod_df_clean, sales_data_df_clean, prod_sales)

,n_cols,n_rows
left_df,7,67175
right_df,5,4526182
merged_df,11,4526182


# Understand the overall market share of our client

In [96]:
# prod_sales.info()
vendor_sales_sum = prod_sales.groupby('vendor')['sales_dollars_value'].sum()
total_sales = vendor_sales_sum.sum()
vendor_sales_percentage = (vendor_sales_sum / total_sales) * 100
vendor_sales_percentage.info()

<class 'pandas.core.series.Series'>
Index: 9 entries, A to Private Label
Series name: sales_dollars_value
Non-Null Count  Dtype  
--------------  -----  
9 non-null      float64
dtypes: float64(1)
memory usage: 144.0+ bytes


In [97]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
bar_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']
plt.bar(vendor_sales_percentage.index, vendor_sales_percentage.values,color = bar_colors)
plt.xlabel('Vendors',fontsize = 15)
plt.ylabel('Sales Percentage',fontsize = 15)
plt.grid(False)
# plt.xticks(fontweight='bold')
# plt.title('Vendor Sales Percentage (Bar Chart)')
# plt.pie(vendor_sales_percentage, labels=vendor_sales_percentage.index, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 20})
plt.title('Percentage of Sales by Vendor',fontsize= 20)
# plt.axis('equal')  # Equal aspect ratio ensures that the pie chart is drawn as a circle.
plt.tight_layout()

plt.savefig('overall market share.png', bbox_inches='tight')


# potential competitors for our client in each theme

In [98]:
# for low carb theme
product_id_for_low_carb = list(set(theme_product_list_df_clean[theme_product_list_df_clean['claim_id'] == 8]['product_id'].tolist()))
prod_sales_low_carb = prod_sales[prod_sales['product_id'].isin(product_id_for_low_carb)]
vendor_sales_low_carb = prod_sales_low_carb.groupby('vendor')['sales_dollars_value'].sum().sort_values(ascending=False)


In [99]:

plt.figure(figsize=(8, 8))

light_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

bars = plt.bar(vendor_sales_low_carb.index, vendor_sales_low_carb.values, color =light_colors)

for bar in bars:
    height = bar.get_height()
    percentage = height / vendor_sales_low_carb.sum() * 100
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{percentage:.2f}%',
             ha='center', va='bottom', fontsize=12, fontweight='bold')
plt.grid(False)
plt.xticks(fontsize = 15,rotation = 45)
plt.tight_layout()
plt.xlabel('Vendors')
plt.ylabel('Sales')
plt.title('Low carb Sales by Vendor',fontsize = 20)
plt.savefig('vendor_sales_low_carb.png', bbox_inches='tight')

In [100]:
# plt.figure(figsize=(10, 6))
# vendor_sales_low_carb.plot(kind='bar', color = ['blue', 'green', 'orange', 'red', 'purple'])
# plt.title('Total Sales in Dollars by Vendor (Low Carb Products)',fontsize = 20)
# plt.xlabel('Vendor',fontsize = 20)
# plt.ylabel('Total Sales Dollars',fontsize = 20)
# plt.xticks(rotation=45, ha='right')

# plt.tight_layout()  # To ensure labels and ticks are visible properly.
# plt.savefig('vendor_sales_low_carb.png', bbox_inches='tight')

### 30 common themes preset in all 3 datasets 

In [101]:
common_themes = ['convenience - easy-to-prepare','nuts','low carb','french bisque',
 'prebiotic',
 'peach',
 'ethical - not specific',
 'soy foods',
 'no additives/preservatives',
 'sea salt',
 'vegetarian',
 'crab',
 'ethnic & exotic',
 'gmo free',
 'low sodium',
 'high/source of protein',
 'tuna',
 'bone health',
 'low calorie',
 'gingerbread',
 'blueberry',
 'mackerel',
 'poultry',
 'chicken',
 'ethical - packaging',
 'beef hamburger',
 'energy/alertness',
 'low carb',
 'halal',
 'salmon',
 'low sugar']
theme_neighbours =[]
for theme in common_themes:
    theme_id = theme_list_df_clean[theme_list_df_clean['claim_name'] == theme]['claim_id'].iloc[0]
    # for each  theme
    product_ids = list(set(theme_product_list_df_clean[theme_product_list_df_clean['claim_id'] == theme_id]['product_id'].tolist()))
    prod_sales_theme = prod_sales[prod_sales['product_id'].isin(product_ids)]

    p_s_t_group = prod_sales_theme.groupby('vendor',as_index = False)['sales_dollars_value'].sum()
    total = p_s_t_group['sales_dollars_value'].sum()
    p_s_t_group['percentage_sales'] = (p_s_t_group['sales_dollars_value'] / int(total)) * 100
    p_s_t_group =p_s_t_group.sort_values('percentage_sales',ascending=False).reset_index().drop(columns = ['index'],axis =1)
    # print(p_s_t_group)
    # vendor_sales_theme = prod_sales_theme.groupby('vendor',as_index = False)['sales_dollars_value'].sum().sort_values('sales_dollars_value',ascending=False).reset_index().drop(columns = ['index'],axis =1)
    # print(vendor_sales_theme)

    vendor_A = 'A'
    if vendor_A not in p_s_t_group['vendor'].values:
        continue
    # Step 4: Find the total sales dollars for vendor A
    # percentage_sales_vendor_A = p_s_t_group.loc[p_s_t_group['vendor'] == vendor_A, 'percentage_sales'].values[0]

    # Step 5: Calculate the threshold for 10% difference
    threshold =  10
    # Step 6: Find the vendors within the +- 10% range of vendor A's sales
    neighbors = p_s_t_group
        # (p_s_t_group['percentage_sales'] >= percentage_sales_vendor_A - threshold) &
        # (p_s_t_group['percentage_sales'] <= percentage_sales_vendor_A + threshold) &
        # (p_s_t_group['vendor'] != vendor_A)
    # ]
    # print(neighbors)
    # position_vendor_A = vendor_sales_theme.index[vendor_sales_theme['vendor'] == vendor_A].tolist()[0]
    # # # Find the neighboring vendors based on their positions
    # num_neighbors = 1  # You can change this value to get more or fewer neighbors
    # neighbors = vendor_sales_theme.iloc[position_vendor_A - num_neighbors:position_vendor_A + 1]
    vendor_list = neighbors['vendor'].tolist()
    percentage_list = neighbors['percentage_sales'].tolist()
    percentage_list =  list(map(lambda x: round(x, 2), percentage_list))
    # print(neighbors)
    theme_neighbour = {'theme':theme, 'neighbours':vendor_list, 'percentages':percentage_list}
    theme_neighbours.append(theme_neighbour)


theme_neighbours_df = pd.DataFrame(theme_neighbours)
theme_neighbours_df


# percentage show

,theme,neighbours,percentages
0,low carb,"[Others, F, A, D, Private Label, B, H]","[50.2, 13.19, 13.05, 11.24, 8.41, 3.72, 0.18]"
1,french bisque,"[Others, D, B, A, F, Private Label]","[52.29, 34.81, 9.46, 3.2, 0.16, 0.08]"
2,soy foods,"[A, B, H, D, F, Others, Private Label]","[53.23, 20.63, 9.67, 6.48, 5.51, 2.79, 1.68]"
3,no additives/preservatives,"[F, Others, B, A, D, Private Label, H]","[33.23, 27.82, 16.54, 11.77, 9.78, 0.5, 0.35]"
4,vegetarian,"[Others, B, A, H]","[75.53, 24.31, 0.13, 0.03]"
5,crab,"[Others, F, A, Private Label]","[81.45, 14.16, 4.33, 0.07]"
6,ethnic & exotic,"[Others, A, B, F, H, D, Private Label]","[34.03, 21.62, 18.12, 10.86, 7.78, 7.56, 0.03]"
7,gmo free,"[F, Others, A, B, Private Label, H]","[64.11, 34.83, 0.59, 0.42, 0.03, 0.02]"
8,low sodium,"[Others, A, B, F, Private Label]","[75.11, 12.99, 6.9, 3.9, 1.1]"
9,high/source of protein,"[Others, A, B]","[79.38, 20.58, 0.04]"


In [102]:
theme_ids = theme_list_df_clean[theme_list_df_clean['claim_name'].isin(common_themes)]['claim_id'].tolist()
# for each  theme
product_ids = list(set(theme_product_list_df_clean[theme_product_list_df_clean['claim_id'].isin(theme_ids)]['product_id'].tolist()))
prod_sales_t = prod_sales[prod_sales['product_id'].isin(product_ids)]
prod_sales_t
prod_sales_theme = pd.merge(prod_sales_t, theme_product_list_df_clean, how='inner', on='product_id', validate='m:m')
merge_info(prod_sales_t, theme_product_list_df_clean, prod_sales_theme)

,n_cols,n_rows
left_df,11,2237206
right_df,2,91485
merged_df,12,5321053


In [103]:
theme_list_df_clean[theme_list_df_clean['claim_id'].isin([8,158,432])]['claim_name'].tolist()

['low carb', 'chicken', 'apple cinnamon']

In [104]:


# prod_sales_theme.dropna(subset=['sales_dollars_value'], inplace=True)  # Remove rows with NaN values
# to add theme names
prod_sales_theme_name = pd.merge(prod_sales_theme, theme_list_df_clean, how='inner', on='claim_id', validate='m:1')
# prod_sales_theme_name
grouped_data = prod_sales_theme_name[['claim_name','vendor','sales_dollars_value']].groupby(['claim_name','vendor'])['sales_dollars_value'].sum().reset_index()
grouped_data = grouped_data[grouped_data['claim_name'].isin(['low carb', 'french bisque', 'soy foods',
                                                             'vegetarian','crab','low sugar','beef hamburger','ethnic & exotic',
                                                             'tuna','salmon','low sodium','high/source of protein',
                                                             'blueberry','poultry','chicken','ethical - packaging','gingerbread',
                                                             'no additives/preservatives'])]

grouped_data['sales_percentage'] = grouped_data.groupby('claim_name')['sales_dollars_value'].transform(lambda x: x / x.sum() * 100)

# Assuming you have 'grouped_data' DataFrame available

# Set the style for the plot (optional but makes it look better)
sns.set(style="whitegrid")

# Create the grouped bar plot using Seaborn
plt.figure(figsize=(15, 10))  # Adjust the figure size as needed

# Replace 'x', 'y', and 'hue' with the appropriate column names from 'grouped_data'
# 'x' should be the x-axis labels, 'y' should be the y-axis values, and 'hue' should be used for grouping.
color_palette = sns.color_palette("Set1", n_colors=len(grouped_data['vendor'].unique()))

# sns.barplot(x='claim_name', y='sales_percentage', hue='vendor', data=grouped_data, palette=color_palette,dodge = True, stacked = True)

grouped_data.drop('sales_dollars_value', axis = 1, inplace  =True)
# grouped_data.set_index('claim_name', inplace = True)
grouped_data = grouped_data.pivot(index='claim_name', columns='vendor', values='sales_percentage')

grouped_data.plot(kind = 'bar',stacked=True,color = color_palette )

# Add labels and title
plt.xlabel('Claim Name')
plt.ylabel('Percentage of Sales')
plt.title('Sales percentage by Claim Name and Vendor', fontsize = 20)

# Rotate the x-axis labels for better visibility (optional)
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.legend(title='Vendor', loc='upper left',bbox_to_anchor=(1.05, 1),fontsize=7)  # Add the legend with the 'Vendor' title
plt.tight_layout()  # Adjust layout to prevent label cutoff
# plt.show()

plt.savefig('grouped_barpng', bbox_inches='tight')

In [105]:
grouped_data

vendor,A,B,D,F,H,Others,Private Label
claim_name,,,,,,,
beef hamburger,2.530518,NaN,NaN,NaN,NaN,97.469482,NaN
blueberry,14.526063,0.106496,0.232452,2.434688,0.012453,73.322303,9.365545
chicken,12.434052,1.160096,5.506604,4.254007,0.400418,69.889802,6.355021
crab,4.329444,NaN,NaN,14.157358,NaN,81.445566,0.067632
ethical - packaging,0.959660,94.143559,NaN,1.779419,NaN,3.117362,NaN
ethnic & exotic,21.616706,18.121477,7.558611,10.862718,7.778449,34.032733,0.029306
french bisque,3.203911,9.455724,34.807366,0.160551,NaN,52.287795,0.084652
gingerbread,5.184596,39.592633,0.528195,6.868711,14.240580,33.585285,NaN
high/source of protein,20.578270,0.040200,NaN,NaN,NaN,79.381529,NaN


### salmon theme

In [106]:
# for beans theme
product_id_for_slamon = list(set(theme_product_list_df_clean[theme_product_list_df_clean['claim_id'] == 227]['product_id'].tolist()))
prod_sales_salmon = prod_sales[prod_sales['product_id'].isin(product_id_for_slamon)]
vendor_sales_salmon = prod_sales_salmon.groupby('vendor')['sales_dollars_value'].sum().sort_values(ascending=False)

In [107]:
plt.figure(figsize=(8, 8))
# plt.pie(vendor_sales_salmon, labels=vendor_sales_salmon.index, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 20})
# plt.title('Total Sales in Dollars by Vendor (salmon Products)',fontsize= 20)
# plt.axis('equal')  # Equal aspect ratio ensures that the pie chart is drawn as a circle.
light_colors = ['#2ca02c', '#d62728', '#9467bd','#1f77b4',   '#8c564b', '#e377c2','#ff7f0e']

bars = plt.bar(vendor_sales_salmon.index, vendor_sales_salmon.values, color =light_colors)

for bar in bars:
    height = bar.get_height()
    percentage = height / vendor_sales_salmon.sum() * 100
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{percentage:.2f}%',
             ha='center', va='bottom', fontsize=12, fontweight='bold')
plt.grid(False)
plt.xlabel('Vendors')
plt.ylabel('Sales')
plt.title('Salmon Sales by Vendor',fontsize = 20)
plt.xticks(fontsize = 15,rotation = 45)
plt.tight_layout()
plt.savefig('vendor_sales_salmon.png', bbox_inches='tight')

In [108]:
# plt.figure(figsize=(10, 6))
# vendor_sales_salmon.plot(kind='pie',fontsize = 18)
# plt.title('Total Sales in Dollars by Vendor (Beans Products)',fontsize = 10)
# # plt.xlabel('Vendor',fontsize = 20)
# # plt.ylabel('Total Sales Dollars',fontsize = 20)
# plt.xticks(rotation=45, ha='right')

# plt.tight_layout()  # To ensure labels and ticks are visible properly.
# plt.savefig('vendor_sales_salmon.png', bbox_inches='tight')

### What are the themes which are emerging in social media, Google Search & Sales?

### emerging percentage bw one year to another

In [109]:
# social_media_data_df_clean.info()
grouped_data = social_media_data_df_clean[social_media_data_df_clean['year']> 2016].groupby(['theme_id','year'])['total_post'].sum().reset_index()


# Create a separate line graph for each 'theme_id'
unique_themes = grouped_data['theme_id'].unique()

plt.figure(figsize=(10, 6))  # Optional: Adjust the size of the figure

# Plotting a line for each 'theme_id'
for theme_id in unique_themes:
    data_for_theme = grouped_data[grouped_data['theme_id'] == theme_id]
    plt.plot(data_for_theme['year'], data_for_theme['total_post'], label=f'Theme {theme_id}')

plt.xlabel('Year')
plt.ylabel('Total Posts')
plt.title('Total Number of Posts by Theme and Year')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()  # Optional: Adjust the layout for better visualization
# plt.savefig('emerging theme', bbox_inches='tight')

Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.


In [110]:
# Find the starting and ending years for each theme
starting_year = grouped_data.groupby('theme_id')['year'].min()
ending_year = grouped_data.groupby('theme_id')['year'].max()
# print(starting_year)
# Calculate the total posts for the starting and ending years for each theme
starting_total_posts = grouped_data[grouped_data['year'] == starting_year.iloc[0]]['total_post']
middle_total_posts = grouped_data[grouped_data['year'] == 2018]['total_post']
ending_total_posts = grouped_data[grouped_data['year'] == ending_year.iloc[0]]['total_post']

# Calculate the percentage increase for each theme
percentage_increase = ((ending_total_posts.values - starting_total_posts.values) / starting_total_posts.values) * 100

# Create a new DataFrame to store the results
result_df = pd.DataFrame({
    'theme_id': starting_year.index,
    'starting_year': starting_year.values,
    'ending_year': ending_year.values,
    'starting_total_posts': starting_total_posts.values,
    'middle_total_posts': middle_total_posts,
    'ending_total_posts': ending_total_posts.values,
    'percentage_increase': percentage_increase
})

result_df.sort_values('percentage_increase',ascending = False, inplace = True)
result_df = result_df.merge(theme_list_df_clean, left_on= 'theme_id', right_on = 'claim_id', how  = 'inner')
result_df = result_df[result_df['claim_name'].isin(common_themes)]
result_df#[['claim_name','percentage_increase']]

divide by zero encountered in true_divide
invalid value encountered in true_divide


,theme_id,starting_year,ending_year,starting_total_posts,middle_total_posts,ending_total_posts,percentage_increase,claim_id,claim_name
12,191,2017,2019,149068,249659,261561,75.464218,191,bone health
15,211,2017,2019,830,1445,1395,68.072289,211,halal
16,125,2017,2019,31054,38667,51915,67.176531,125,nuts
20,192,2017,2019,2165,3450,3388,56.489607,192,prebiotic
24,433,2017,2019,26406,39271,38498,45.792623,433,gingerbread
26,120,2017,2019,2979,17392,4213,41.423296,120,crab
38,227,2017,2019,48133,100310,56418,17.212723,227,salmon
39,40,2017,2019,7719,6812,8848,14.626247,40,no additives/preservatives
43,65,2017,2019,13906,23947,15080,8.442399,65,ethical - not specific
46,393,2017,2019,8445,48858,9066,7.353464,393,sea salt


### themes which are emerging in social media for year 2018

In [111]:
social_18 = social_media_data_df_clean[social_media_data_df_clean['year'].isin([2018,2019])].sort_values('published_date')
# common themes
common_theme_ids = theme_list_df_clean[theme_list_df_clean['claim_name'].isin(common_themes)]['claim_id'].tolist()
social_18 = social_18[social_18['theme_id'].isin(common_theme_ids)]


# Filter data for the beginning of the year (January 1, 2017)
start_of_year = social_18[social_18['published_date'] == '2018-01-01']
# Filter data for the end of the year (December 31, 2017)
end_of_year = social_18[social_18['published_date'] == '2019-12-10']

# Group the DataFrame by 'theme_id' and calculate the total posts for each theme
total_posts_start = start_of_year.groupby('theme_id')['total_post'].sum()
total_posts_end = end_of_year.groupby('theme_id')['total_post'].sum()

# Calculate the percentage increase for each theme
percentage_increase = ((total_posts_end - total_posts_start) / total_posts_start) * 100
change = total_posts_end - total_posts_start
start  = total_posts_start
# print(percentage_increase)

# Create a new DataFrame to store the results
result_df = pd.DataFrame({
    'theme_id': percentage_increase.index,
    'percentage_increase': percentage_increase.values,
    'change': change.values,
    'start': start.values

})
result_df.sort_values('percentage_increase',ascending = False, inplace = True)
result_df = result_df.merge(theme_list_df_clean, left_on= 'theme_id', right_on = 'claim_id', how  = 'inner')
result_df = result_df[result_df['change'] > 10]
result_df['percentage_increase'] =  list(map(lambda x: round(x, 2), result_df['percentage_increase']))
result_df[['claim_name','percentage_increase','change','start']].head(15)

,claim_name,percentage_increase,change,start
0,sea salt,688.89,62,9
1,high/source of protein,566.67,17,3
3,chicken,360.66,220,61
4,gingerbread,100.00,50,50
6,salmon,50.00,16,32
8,bone health,28.00,126,450


In [112]:
import numpy as np

def replace_outliers_with_adjacent_mean(x):
    # Set the threshold for identifying outliers (e.g., z-score > 3 or z-score < -3)
    mean_x = np.mean(x)
    std_x = np.std(x)
    
    # Set the threshold for identifying outliers (e.g., z-score > 3 or z-score < -3)
    threshold = 3.0
    
    # Create a mask to identify outliers
    outliers_mask = np.abs((x - mean_x) / std_x) > threshold
    
    # Replace outliers with the mean
    x[outliers_mask] = mean_x

    return x


### Total posts for seeds theme graph -- social

In [113]:
from scipy.signal import savgol_filter
social_1819 = social_media_data_df_clean[social_media_data_df_clean['year'].isin([2018,2019])].sort_values('published_date')

social_1819_bone = social_1819[social_1819['theme_id'] == 191]
social_1819_bone['week'] = social_1819_bone['published_date'].dt.isocalendar().week      # .apply(lambda x : x.isocalendar()[1])
social_1819_bone['year'] = social_1819_bone['published_date'].dt.year

social_1819_bone_date = social_1819_bone.groupby(['published_date'])['total_post'].sum()

# social_1819_bone_date_smooth = replace_outliers_with_adjacent_mean(social_1819_bone_date.values)
social_1819_bone_date_smooth = savgol_filter(social_1819_bone_date.values, window_length=40, polyorder=1)
plt.figure(figsize=(10, 6))  

# Plot the data as a line graph
plt.plot(social_1819_bone_date.index, social_1819_bone_date_smooth, linestyle='-')

# Set labels and title
plt.xlabel('date')
plt.grid(False)
# plt.ylim(0,1000)
plt.ylabel('Total posts')
plt.title('Total posts for Bone health theme', fontsize = 20)
plt.savefig('Total posts for Bone health theme', bbox_inches='tight')

In [114]:
theme_list_df_clean[theme_list_df_clean['claim_name'] == 'low carb']

,claim_id,claim_name
1,8,low carb


### themes which are emerging in Google Search  for 2018

In [115]:
# google_search_data_df_clean.info()
google_18 = google_search_data_df_clean[google_search_data_df_clean['year'].isin([2018,2019])].sort_values('date')
# common themes
common_theme_ids = theme_list_df_clean[theme_list_df_clean['claim_name'].isin(common_themes)]['claim_id'].tolist()
google_18 = google_18[google_18['claim_id'].isin(common_theme_ids)]

# Filter data for the beginning of the year (January 1, 2017)
start_of_year = google_18[google_18['date'] == '2018-01-01']
# Filter data for the end of the year (December 31, 2017)
end_of_year = google_18[google_18['date'] == '2019-12-9']

# Group the DataFrame by 'theme_id' and calculate the total posts for each theme
total_posts_start = start_of_year.groupby('claim_id')['search_volume'].sum()
total_posts_end = end_of_year.groupby('claim_id')['search_volume'].sum()

# Calculate the percentage increase for each theme
percentage_increase = ((total_posts_end - total_posts_start) / total_posts_start) * 100
change = total_posts_end - total_posts_start
start  = total_posts_start
# print(percentage_increase)

# Create a new DataFrame to store the results
result_df = pd.DataFrame({
    'theme_id': percentage_increase.index,
    'percentage_increase': percentage_increase.values,
    'change':change,
    'start': start
})
result_df.sort_values('percentage_increase',ascending = False, inplace = True)
result_df = result_df.merge(theme_list_df_clean, left_on= 'theme_id', right_on = 'claim_id', how  = 'inner')
result_df = result_df[result_df['change'] > 500]
result_df['percentage_increase'] =  list(map(lambda x: round(x, 2), result_df['percentage_increase']))
result_df[['claim_name','percentage_increase','change','start']].head(15)

,claim_name,percentage_increase,change,start
0,bone health,1340.74,1086.0,81.0
1,low sodium,241.81,7530.0,3114.0
2,soy foods,226.58,537.0,237.0
3,vegetarian,153.30,2505.0,1634.0
4,chicken,83.24,4277.0,5138.0
5,french bisque,80.34,760.0,946.0
6,beef hamburger,50.23,542.0,1079.0
7,blueberry,35.92,1324.0,3686.0


In [116]:
google_18['date'].max()


Timestamp('2019-12-09 00:00:00')

### Total searches for bone health theme graph -- google

In [117]:
# from scipy.signal import savgol_filter
google_1819 = google_search_data_df_clean[google_search_data_df_clean['year'].isin([2018,2019])].sort_values('date')

google_1819_bone = google_1819[google_1819['claim_id'] == 191]
google_1819_bone['week'] = google_1819_bone['date'].apply(lambda x : x.isocalendar()[1])
google_1819_bone_week = google_1819_bone.groupby('date')['search_volume'].sum()

google_1819_bone_week_smooth = savgol_filter(google_1819_bone_week.values, window_length=30, polyorder=1)

plt.figure(figsize=(10, 6))  

# Plot the data as a line graph
plt.plot(google_1819_bone_week.index, google_1819_bone_week_smooth, linestyle='-')

# Set labels and title
plt.xlabel('days')
plt.ylabel('Total searches')
plt.title('Total searches for bone health theme', fontsize = 20)
plt.savefig('Total searches for bone health theme', bbox_inches='tight')

### themes which are emerging in sales  for 2018

In [118]:

sales_theme_product = pd.merge(sales_data_df_clean[sales_data_df_clean['system_calendar_key_n'].dt.year == 2018], theme_product_list_df_clean, on = 'product_id',  how='inner', validate='m:m')
merge_info(sales_data_df_clean[sales_data_df_clean['system_calendar_key_n'].dt.year == 2018],  theme_product_list_df_clean,sales_theme_product)
# sales_data_df_clean.info()

,n_cols,n_rows
left_df,5,1248368
right_df,2,91485
merged_df,6,2188399


In [119]:
sales_data_df_clean['year'] = sales_data_df_clean['system_calendar_key_n'].dt.year
sales_theme_product = pd.merge(sales_data_df_clean[sales_data_df_clean['year'].isin([2018,2019])], theme_product_list_df_clean, on = 'product_id',  how='inner', validate='m:m')
merge_info(sales_data_df_clean[sales_data_df_clean['year'].isin([2018,2019])],  theme_product_list_df_clean,sales_theme_product)
# sales_data_df_clean.info()

,n_cols,n_rows
left_df,6,2187087
right_df,2,91485
merged_df,7,3854785


In [120]:

sales_18 = sales_theme_product[sales_theme_product['year'].isin([2018,2019])].sort_values('system_calendar_key_n')

# common themes
common_theme_ids = theme_list_df_clean[theme_list_df_clean['claim_name'].isin(common_themes)]['claim_id'].tolist()
sales_18 = sales_18[sales_18['claim_id'].isin(common_theme_ids)]

# Filter data for the beginning of the year (January 1, 2017)
start_of_year = sales_18[sales_18['system_calendar_key_n'] == '2018-01-06']
# Filter data for the end of the year (December 31, 2017)
end_of_year = sales_18[sales_18['system_calendar_key_n'] == '2019-10-05']
# print(start_of_year)

# Group the DataFrame by 'theme_id' and calculate the total posts for each theme
total_posts_start = start_of_year.groupby('claim_id')['sales_dollars_value'].sum()
total_posts_end = end_of_year.groupby('claim_id')['sales_dollars_value'].sum()

# Calculate the percentage increase for each theme
percentage_increase = ((total_posts_end - total_posts_start) / total_posts_start) * 100
change = total_posts_end - total_posts_start
start  = total_posts_start
# print(percentage_increase)

# Create a new DataFrame to store the results
result_df = pd.DataFrame({
    'theme_id': percentage_increase.index,
    'percentage_increase': percentage_increase.values,
    'change': change,
    'start': start
})
result_df.sort_values('percentage_increase',ascending = False,inplace = True)
result_df = result_df.merge(theme_list_df_clean, left_on= 'theme_id', right_on = 'claim_id', how  = 'inner')
result_df = result_df[result_df['change'] > 18000]
result_df['percentage_increase'] =  list(map(lambda x: round(x, 2), result_df['percentage_increase']))
result_df[['claim_name','percentage_increase','change','start']].head(15)

,claim_name,percentage_increase,change,start
0,bone health,1156.13,18845.0,1630.0
1,poultry,494.46,1101027.0,222673.0
2,low sugar,353.96,344071.0,97207.0
3,beef hamburger,277.54,31004.0,11171.0
4,vegetarian,191.10,708409.0,370700.0
5,convenience - easy-to-prepare,118.15,86697.0,73380.0
6,low sodium,62.19,1707801.0,2746232.0
7,nuts,55.73,119828.0,215031.0
8,prebiotic,54.12,29779.0,55025.0
9,chicken,52.61,796523.0,1514136.0


In [121]:
sales_18['system_calendar_key_n'].max()

Timestamp('2019-10-05 00:00:00')

### Total sales for bone health theme graph -- sales

In [122]:
# from scipy.signal import savgol_filter
sales_theme_product['year'] = sales_theme_product['system_calendar_key_n'].dt.year
sales_1819 = sales_theme_product[sales_theme_product['year'].isin([2018,2019])].sort_values('system_calendar_key_n')

sales_1819_bone = sales_1819[sales_1819['claim_id'] == 228]
sales_1819_bone['week'] = sales_1819_bone['system_calendar_key_n'].apply(lambda x : x.isocalendar()[1])
sales_1819_bone_date = sales_1819_bone.groupby('system_calendar_key_n')['sales_dollars_value'].sum()

sales_1819_bone_date_smooth = savgol_filter(sales_1819_bone_date.values, window_length=5, polyorder=1)

plt.figure(figsize=(10, 6))  

# Plot the data as a line graph
plt.plot(sales_1819_bone_date.index, sales_1819_bone_date_smooth, linestyle='-')

# Set labels and title
plt.xlabel('date')
plt.ylabel('Total sales_dollars_value')
plt.grid(False)
plt.title('Total sales_dollars_value for beef hamburger theme', fontsize = 20)
plt.savefig('Total sales_dollars_value for beef hamburger theme', bbox_inches='tight')

In [123]:
sales_theme_product.head()

,system_calendar_key_n,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,year,claim_id
0,2018-01-06,10756,5113,5147,965,2018,81
1,2018-01-06,10756,5113,5147,965,2018,8
2,2018-01-20,10756,5819,5861,1099,2018,81
3,2018-01-20,10756,5819,5861,1099,2018,8
4,2018-01-13,10756,4234,4256,798,2018,81


#  aggregation

### sales data

In [124]:
# only themes which are common in 3 data sets are considered
common_theme_ids = theme_list_df_clean[theme_list_df_clean['claim_name'].isin(common_themes)]['claim_id'].tolist()

# merge bw sales and prod manufacturer list --> adds vendor column
sales_manufacture = pd.merge(sales_data_df_clean, prod_df_clean, on = 'product_id',  how='inner', validate='m:1')
sales_manufacture_A = sales_manufacture[sales_manufacture['vendor'] == 'A']

# theme product list only for common theme ids
theme_product_list_df_clean_common = theme_product_list_df_clean[theme_product_list_df_clean['claim_id'].isin(common_theme_ids)]

# merged the data bw sales with vendor and common theme product list to include claim id
sales_manufacture_theme = pd.merge(sales_manufacture_A, theme_product_list_df_clean_common, on = 'product_id',  how='inner', validate='m:m')
merge_info(sales_manufacture,  theme_product_list_df_clean_common, sales_manufacture_theme)

,n_cols,n_rows
left_df,12,4526182
right_df,2,39331
merged_df,13,338918


In [125]:
# aggrigate the sales  data in weekly basis
sales_manufacture_theme['week'] = sales_manufacture_theme['system_calendar_key_n'].apply(lambda x : x.isocalendar()[1])
sales_manufacture_theme['year'] = sales_manufacture_theme['system_calendar_key_n'].dt.year
sales_manufacture_theme_week = sales_manufacture_theme.drop(['system_calendar_key_n','product_id'],axis = 1)
sales_manufacture_theme_week = sales_manufacture_theme_week.groupby(['claim_id','year','week'],as_index = False).agg(
    sales_dollars_value = ('sales_dollars_value','sum'),
    sales_units_value = ('sales_units_value','sum'),
    sales_lbs_value = ('sales_lbs_value','sum') )

sales_manufacture_theme_week.head()

,claim_id,year,week,sales_dollars_value,sales_units_value,sales_lbs_value
0,8,2016,1,8853853,1952575,2947044
1,8,2016,2,8705721,1921171,2901409
2,8,2016,3,8839233,1968251,2954048
3,8,2016,4,8872568,1974980,2941180
4,8,2016,5,9166248,2092314,3012709


In [126]:
# sales_manufacture_theme_week['claim_id'].value_counts()

### social media data

In [127]:
social_media_data_df_clean_common = social_media_data_df_clean[social_media_data_df_clean['theme_id'].isin(common_theme_ids)]
import datetime
# aggrigate the social media  data in weekly basis
social_media_data_df_clean_common['published_date'] = social_media_data_df_clean_common['published_date'] + datetime.timedelta(days=21)
social_media_data_df_clean_common['week'] = social_media_data_df_clean_common['published_date'].apply(lambda x : x.isocalendar()[1])
social_media_data_df_clean_common['year'] = social_media_data_df_clean_common['published_date'].dt.year
social_media_data_df_clean_common_week = social_media_data_df_clean_common.drop(['published_date'],axis = 1)
social_media_data_df_clean_common_week = social_media_data_df_clean_common_week.groupby(['theme_id','year','week'],as_index = False).agg(
    total_post = ('total_post','sum') )

social_media_data_df_clean_common_week.head()

,theme_id,year,week,total_post
0,8,2015,5,1041
1,8,2015,6,123
2,8,2015,9,406
3,8,2015,10,1042
4,8,2015,13,429


In [128]:
social_media_data_df_clean_common_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7367 entries, 0 to 7366
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   theme_id    7367 non-null   int64
 1   year        7367 non-null   int64
 2   week        7367 non-null   int64
 3   total_post  7367 non-null   int64
dtypes: int64(4)
memory usage: 230.3 KB


### google search

In [129]:
google_search_data_df_clean_common = google_search_data_df_clean[google_search_data_df_clean['claim_id'].isin(common_theme_ids)]

# aggrigate the social media  data in weekly basis
google_search_data_df_clean_common['date'] = google_search_data_df_clean_common['date']  + datetime.timedelta(days=7)
google_search_data_df_clean_common['week'] = google_search_data_df_clean_common['date'].apply(lambda x : x.isocalendar()[1])
google_search_data_df_clean_common['year'] = google_search_data_df_clean_common['date'].dt.year
google_search_data_df_clean_common_week = google_search_data_df_clean_common.drop(['date','platform','year_new','week_number'],axis = 1)
google_search_data_df_clean_common_week = google_search_data_df_clean_common_week.groupby(['claim_id','year','week'],as_index = False).agg(
    search_volume = ('search_volume','sum') )

google_search_data_df_clean_common_week.head()

,claim_id,year,week,search_volume
0,8,2014,1,16659
1,8,2014,2,26833
2,8,2014,3,40695
3,8,2014,4,54171
4,8,2014,5,39676


In [130]:
google_search_data_df_clean_common_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6408 entries, 0 to 6407
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   claim_id       6408 non-null   int64
 1   year           6408 non-null   int64
 2   week           6408 non-null   int64
 3   search_volume  6408 non-null   int64
dtypes: int64(4)
memory usage: 200.4 KB


## merge

### merge bw social and google data

In [131]:

social_google = pd.merge(social_media_data_df_clean_common_week,google_search_data_df_clean_common_week, how='inner', 
left_on=['theme_id','year','week'],right_on=['claim_id','year','week'], validate='1:1')

merge_info(social_media_data_df_clean_common_week, google_search_data_df_clean_common_week, social_google)

,n_cols,n_rows
left_df,4,7367
right_df,4,6408
merged_df,6,5135


In [132]:
social_google.drop(['theme_id'],axis = 1, inplace  = True)
social_google.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5135 entries, 0 to 5134
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   year           5135 non-null   int64
 1   week           5135 non-null   int64
 2   total_post     5135 non-null   int64
 3   claim_id       5135 non-null   int64
 4   search_volume  5135 non-null   int64
dtypes: int64(5)
memory usage: 240.7 KB


### merge bw social_google and sales data

In [133]:
social_google_sales = pd.merge(social_google,sales_manufacture_theme_week, how='inner', 
on=['claim_id','year','week'], validate='1:1')
merge_info(social_google, sales_manufacture_theme_week, social_google_sales)

,n_cols,n_rows
left_df,5,5135
right_df,6,3231
merged_df,8,2630


# final data after merge

In [134]:
social_google_sales.head()

,year,week,total_post,claim_id,search_volume,sales_dollars_value,sales_units_value,sales_lbs_value
0,2016,1,983,8,49214,8853853,1952575,2947044
1,2016,2,726,8,46543,8705721,1921171,2901409
2,2016,3,1180,8,44716,8839233,1968251,2954048
3,2016,4,2014,8,53664,8872568,1974980,2941180
4,2016,5,1586,8,47560,9166248,2092314,3012709


In [135]:
save_dataset(context, social_google_sales, 'processed/social_google_sales')

In [136]:
# week_number and week are different
google_search_data_df_clean_common[google_search_data_df_clean_common['week_number'] != google_search_data_df_clean_common['week']].head()

,date,claim_id,platform,year_new,week_number,search_volume,year,week
0,2014-01-08,8,google,2014,1,6613,2014,2
1,2014-01-08,39,google,2014,1,181,2014,2
2,2014-01-08,75,google,2014,1,135,2014,2
3,2014-01-08,81,google,2014,1,1257,2014,2
4,2014-01-08,100,google,2014,1,2636,2014,2


In [137]:
sales_manufacture_theme_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3231 entries, 0 to 3230
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   claim_id             3231 non-null   int64
 1   year                 3231 non-null   int64
 2   week                 3231 non-null   int64
 3   sales_dollars_value  3231 non-null   int64
 4   sales_units_value    3231 non-null   int64
 5   sales_lbs_value      3231 non-null   int64
dtypes: int64(6)
memory usage: 151.6 KB


In [138]:
prod_sales.head()

,product_id,vendor,unnamed_2,unnamed_3,unnamed_4,unnamed_5,unnamed_6,system_calendar_key_n,sales_dollars_value,sales_units_value,sales_lbs_value
0,1,Others,NaN,NaN,NaN,NaN,NaN,2016-01-09,13927,934,18680
1,1,Others,NaN,NaN,NaN,NaN,NaN,2016-01-23,12628,878,17564
2,1,Others,NaN,NaN,NaN,NaN,NaN,2016-02-06,11379,810,16200
3,1,Others,NaN,NaN,NaN,NaN,NaN,2016-01-30,11568,821,16424
4,1,Others,NaN,NaN,NaN,NaN,NaN,2016-02-13,10959,784,15682


In [139]:
social_media_data_df_clean[['theme_id','published_date']].value_counts()
social_media_data_df_clean[(social_media_data_df_clean['theme_id'] == 788) & (social_media_data_df_clean['published_date'] == '30-07-2019')]
# social_media_data_df_clean_t[(social_media_data_df_clean_t['theme_id'] == 788) & (social_media_data_df_clean_t['published_date'] == '30-07-2019')]
social_media_data_df_clean['theme_id'].nunique()

Parsing '30-07-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.


193

In [140]:
# popular theme in social media
df1 = social_media_data_df_clean_16.groupby(['theme_id'], as_index = False)['total_post'].sum().sort_values('total_post', ascending = False)
popular_theme_id_list_social = df1[df1['total_post']> 400000]['theme_id'].tolist()
popular_theme_id_list_social
# df1.head(10)
social_media_data_df_clean_popular_16 = social_media_data_df_clean_16[social_media_data_df_clean_16['theme_id'].isin(popular_theme_id_list_social)]

In [141]:
# popular theme in social to google
google_search_data_df_clean_popular_1617 = google_search_data_df_clean_1617[google_search_data_df_clean_1617['claim_id'].isin(popular_theme_id_list_social)]


In [142]:
google_search_data_df_clean[['date','claim_id','platform','year_new','week_number']].value_counts()
# google_search_data_df_clean.groupby(['date','claim_id','platform','year_new','week_number'])['search_volume'].sum()

date        claim_id  platform  year_new  week_number
2014-01-01  8         google    2014      1              1
2018-07-28  684       amazon    2018      30             1
            668       amazon    2018      30             1
                      chewy     2018      30             1
                      google    2018      30             1
                                                        ..
2016-10-31  908       google    2016      44             1
            918       google    2016      44             1
            922       google    2016      44             1
            929       google    2016      44             1
2019-12-09  999       google    2019      37             1
Length: 179082, dtype: int64

In [143]:
social_and_google = pd.merge(social_media_data_df_clean_popular_16, google_search_data_df_clean_popular_1617, left_on = 'theme_id', right_on = 'claim_id',  how='inner', validate='m:m')
merge_info(social_media_data_df_clean_popular_16,  google_search_data_df_clean_popular_1617,social_and_google)

,n_cols,n_rows
left_df,4,1464
right_df,7,2924
merged_df,11,1070184


In [144]:
social_and_google[social_and_google['published_date'] < social_and_google['date']]
social_and_google.drop('claim_id',axis = 1)

,theme_id,published_date,total_post,year_x,date,platform,year_new,week_number,search_volume,year_y
0,576,2016-01-01,2188,2016,2016-01-01,google,2016,53,25037,2016
1,576,2016-01-01,2188,2016,2017-01-01,google,2017,52,22382,2017
2,576,2016-01-01,2188,2016,2016-01-02,google,2016,5,19076,2016
3,576,2016-01-01,2188,2016,2017-01-02,google,2017,5,28533,2017
4,576,2016-01-01,2188,2016,2016-01-03,google,2016,9,15300,2016
...,...,...,...,...,...,...,...,...,...,...
1070179,770,2016-12-31,968,2016,2017-08-31,google,2017,35,17694,2017
1070180,770,2016-12-31,968,2016,2016-10-31,google,2016,44,11192,2016
1070181,770,2016-12-31,968,2016,2017-10-31,google,2017,44,15774,2017
1070182,770,2016-12-31,968,2016,2016-12-31,google,2017,52,10085,2016


In [145]:
google_search_data_df_clean[google_search_data_df_clean['claim_id'] == 916].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 1370 to 178550
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           372 non-null    datetime64[ns]
 1   claim_id       372 non-null    int64         
 2   platform       372 non-null    object        
 3   year_new       372 non-null    int64         
 4   week_number    372 non-null    int64         
 5   search_volume  372 non-null    int64         
 6   year           372 non-null    int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 23.2+ KB


In [146]:
theme_list_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 0 to 207
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   claim_id    208 non-null    int64 
 1   claim_name  208 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.9+ KB


In [149]:
theme_theme_product = pd.merge(theme_list_df_clean, google_search_data_df_clean, how='inner', left_on='claim_id',right_on = 'claim_id', validate='1:m')
# merge_info(prod_df_clean, sales_data_df_clean, prod_sales)